In [1]:
import numpy as np

In [2]:
'''
Software to test visible space representation presented in
https://journals.sagepub.com/doi/full/10.5772/63484

Dynamic Path planning on Mobile Robot based on Visible Space and Improved GA.
'''

'\nSoftware to test visible space representation presented in\nhttps://journals.sagepub.com/doi/full/10.5772/63484\n\nDynamic Path planning on Mobile Robot based on Visible Space and Improved GA.\n'

In [9]:
map1 = np.zeros(shape=(5,5))
obstacles = np.array([(2,2),(2,3),(2,4),(3,2),(3,3),(3,4),(4,2),(4,3),(4,4)])
for i,j in obstacles:
    map1[i-1,j-1] = 1
map1

array([[0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [41]:
def get_outer_envelope(obstacles_array, map_array):
    transition_points = set()
    row_transition = set()
    col_transition = set()
    for i,j in obstacles:
        i -= 1
        j -= 1
        dir_list = ['left','right','up','down']
        if i == 0:
            dir_list.remove('up')
        if i == map_array.shape[0]-1:
            dir_list.remove('down')
        if j == 0:
            dir_list.remove('left')
        if j == map_array.shape[1]-1:
            dir_list.remove('right')


        for direction in dir_list:
            if direction == 'left':
                if map_array[i,j-1] == 0:
                    row_transition.add( (i+1,j+1) )
            elif direction == 'right':
                if map_array[i,j+1] == 0:
                    row_transition.add( (i+1,j+1) )
            if direction == 'up':
                if map_array[i-1,j] == 0:
                    col_transition.add( (i+1,j+1) )
            elif direction == 'down':
                if map_array[i+1,j] == 0:
                    col_transition.add( (i+1,j+1) )

                    
        transition_points = row_transition & col_transition
    return transition_points

In [42]:
get_outer_envelope(obstacles, map1)

{(2, 2), (2, 4), (4, 2), (4, 4)}